In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from generator_model import Generator
from discriminator_model import Discriminator
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import numpy as np
from glob import glob
import cv2
import matplotlib.pyplot as plt
import os
from scipy.ndimage import gaussian_filter, map_coordinates

torch.backends.cudnn.benchmark = True





In [2]:
def train_fn(
    disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler,
    device='cuda'
):
    loop = tqdm(loader, leave=True)
    
    # Track total losses for averaging
    total_d_loss = 0
    total_g_loss = 0
    total_g_fake_loss = 0
    total_l1_loss = 0

    gen.train()
    disc.train()

    for idx, (x, y) in enumerate(loop):
        x = x.to(device)
        y = y.to(device)

        # ----------- Train Discriminator -----------
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake = disc(x, y_fake.detach())
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # ----------- Train Generator -----------
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, y) * 100
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        # ----------- Logging and Metrics -----------
        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
                D_loss=D_loss.item(),
                G_loss=G_loss.item(),
            )

        # Track for averaging
        total_d_loss += D_loss.item()
        total_g_loss += G_loss.item()
        total_g_fake_loss += G_fake_loss.item()
        total_l1_loss += L1.item()

    # Compute average losses over the epoch
    num_batches = len(loader)
    avg_d_loss = total_d_loss / num_batches
    avg_g_loss = total_g_loss / num_batches
    avg_g_fake_loss = total_g_fake_loss / num_batches
    avg_l1_loss = total_l1_loss / num_batches

    return {
        'D_loss': avg_d_loss,
        'G_loss': avg_g_loss,
        'G_fake_loss': avg_g_fake_loss,
        'L1_loss': avg_l1_loss,
    }

In [3]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torchmetrics import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure, MeanAbsoluteError


def eval_fn(gen, disc, val_loader, l1_loss, bce, device='cuda'):
    gen.eval()
    disc.eval()

    total_l1_loss = 0
    total_g_fake_loss = 0
    total_g_loss = 0

    # Image quality metrics
    mae_metric = MeanAbsoluteError().to(device)
    psnr_metric = PeakSignalNoiseRatio().to(device)
    ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

    loop = tqdm(val_loader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for x, y in loop:
            x = x.to(device)
            y = y.to(device)

            with torch.cuda.amp.autocast():
                y_fake = gen(x)
                D_fake = disc(x, y_fake)
                G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
                L1 = l1_loss(y_fake, y) * 100
                G_loss = G_fake_loss + L1

            total_l1_loss += L1.item()
            total_g_fake_loss += G_fake_loss.item()
            total_g_loss += G_loss.item()

            # Clamp predictions to [0, 1] for metrics
            y_fake_clamped = torch.clamp(y_fake, 0, 1)

            # Update metrics
            mae_metric.update(y_fake_clamped, y)
            psnr_metric.update(y_fake_clamped, y)
            ssim_metric.update(y_fake_clamped, y)

    num_batches = len(val_loader)
    avg_l1_loss = total_l1_loss / num_batches
    avg_g_fake_loss = total_g_fake_loss / num_batches
    avg_g_loss = total_g_loss / num_batches

    # Compute metrics
    mae = mae_metric.compute().item()
    psnr = psnr_metric.compute().item()
    ssim = ssim_metric.compute().item()

    print(f"\n[Validation] G_loss: {avg_g_loss:.4f}, "
          f"G_fake_loss: {avg_g_fake_loss:.4f}, "
          f"L1_loss: {avg_l1_loss:.4f}")
    print(f"[Metrics] MAE: {mae:.4f}, PSNR: {psnr:.2f} dB, SSIM: {ssim:.4f}")

    return {
        'G_loss': avg_g_loss,
        'G_fake_loss': avg_g_fake_loss,
        'L1_loss': avg_l1_loss,
        'MAE': mae,
        'PSNR': psnr,
        'SSIM': ssim,
    }


In [4]:
class MyCustomDataset(Dataset):
    def __init__(self,train_path,test_path,image_size=(224, 224),aug=True):
        self.train_path = train_path
        self.test_path = test_path
        self.image_size = image_size
        self.aug = aug

        self.train_images = sorted(glob(os.path.join(train_path, '*.npy')))
    def __len__(self):
        return len(self.train_images)

    def rotate(self,mr,ct):
        
        h, w = self.image_size
        center = (self.image_size[0] // 2, self.image_size[0] // 2)
        angle = np.random.uniform(-10, 10)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale=1.0)
        mr_rotated = cv2.warpAffine(mr, rotation_matrix, (w, h),borderMode=cv2.BORDER_REFLECT_101)
        ct_rotated = cv2.warpAffine(ct, rotation_matrix, (w, h),borderMode=cv2.BORDER_REFLECT_101)
        return mr_rotated,ct_rotated
        
    def hflip(self,mr,ct):
        mr_flipped = cv2.flip(mr, 1)
        ct_flipped = cv2.flip(ct, 1)
        return mr_flipped,ct_flipped

    def scaled(self, mr, ct):
        h, w = self.image_size
    
        # Random scale factor between 0.9 (zoom out) and 1.1 (zoom in)
        scale = np.random.uniform(0.9, 1.1)
    
        # Resize image
        scaled_mr = cv2.resize(mr, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        scaled_ct = cv2.resize(ct, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    
        # Get new size
        new_h, new_w = scaled_mr.shape[:2]
    
        # Crop or pad to original size (centered)
        top = max((new_h - h) // 2, 0)
        left = max((new_w - w) // 2, 0)
        bottom = top + h
        right = left + w
    
        # If scaled image is larger — crop center
        if scale >= 1.0:
            scaled_mr = scaled_mr[top:bottom, left:right]
            scaled_ct = scaled_ct[top:bottom, left:right]
        else:
            # If scaled image is smaller — pad to original size
            pad_top = (h - new_h) // 2
            pad_bottom = h - new_h - pad_top
            pad_left = (w - new_w) // 2
            pad_right = w - new_w - pad_left
    
            scaled_mr = cv2.copyMakeBorder(scaled_mr, pad_top, pad_bottom, pad_left, pad_right, borderType=cv2.BORDER_REFLECT_101)
            scaled_ct = cv2.copyMakeBorder(scaled_ct, pad_top, pad_bottom, pad_left, pad_right, borderType=cv2.BORDER_REFLECT_101)
    
        return scaled_mr, scaled_ct
        
    def translate(self, mr, ct):
        h, w = self.image_size
    
        # Max shift: 10% of width and height
        max_shift_x = int(0.1 * w)
        max_shift_y = int(0.1 * h)
    
        # Random shifts in x and y directions
        tx = np.random.randint(-max_shift_x, max_shift_x + 1)
        ty = np.random.randint(-max_shift_y, max_shift_y + 1)
    
        # Create translation matrix
        translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    
        # Apply translation
        mr_translated = cv2.warpAffine(mr, translation_matrix, (w, h), borderMode=cv2.BORDER_REFLECT_101)
        ct_translated = cv2.warpAffine(ct, translation_matrix, (w, h), borderMode=cv2.BORDER_REFLECT_101)
    
        return mr_translated, ct_translated

    def elastic_deformation(self,image, dx, dy):
        shape = image.shape[:2]
    
        x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
        indices = np.reshape(y + dy, (-1)), np.reshape(x + dx, (-1))
    
        if image.ndim == 3:
            channels = []
            for c in range(image.shape[2]):
                channel = map_coordinates(image[..., c], indices, order=1, mode='reflect').reshape(shape)
                channels.append(channel)
            return np.stack(channels, axis=-1).astype(image.dtype)
        else:
            return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape).astype(image.dtype)
    
    def elastic(self, mr, ct, alpha=30, sigma=4):
        shape = mr.shape[:2]
        random_state = np.random.RandomState(None)
    
        dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
        dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    
        mr_deformed = self.elastic_deformation(mr, dx, dy)
        ct_deformed = self.elastic_deformation(ct, dx, dy)
    
        return mr_deformed, ct_deformed

    def augment(self, mr, ct):
        # You can tweak these probabilities (0.5 = 50% chance)
        if np.random.rand() < 0.5:
            mr, ct = self.rotate(mr, ct)
        if np.random.rand() < 0.5:
            mr, ct = self.hflip(mr, ct)
        if np.random.rand() < 0.5:
            mr, ct = self.scaled(mr, ct)
        if np.random.rand() < 0.5:
            mr, ct = self.translate(mr, ct)
        # if np.random.rand() < 0.5:
        #     mr, ct = self.elastic(mr, ct)
    
        return mr, ct
    
    def __getitem__(self, idx):
        mr_path = self.train_images[idx]
        filename = os.path.basename(mr_path)

        # CT file (should match filename)
        ct_path = os.path.join(self.test_path, filename)
        # print(mr_path)
        mr_img = np.load(mr_path)
        ct_img = np.load(ct_path)

        if self.image_size is not None:
            mr_img = cv2.resize(mr_img, self.image_size)
            ct_img = cv2.resize(ct_img, self.image_size)
            
        if self.aug:
            mr_img, ct_img = self.augment(mr_img, ct_img)

        if mr_img.ndim == 2:
            mr_img = np.expand_dims(mr_img, axis=-1)
        if ct_img.ndim == 2:
            ct_img = np.expand_dims(ct_img, axis=-1)



        mr_tensor = torch.from_numpy(mr_img).permute(2, 0, 1).float()
        ct_tensor = torch.from_numpy(ct_img).permute(2, 0, 1).float()

        return mr_tensor, ct_tensor


In [5]:
dataTrain = MyCustomDataset('../../ct_mr_stdscale//train/mr/','../../ct_mr_stdscale/train/ct/')
dataVal = MyCustomDataset('../../ct_mr_stdscale/val//mr/','../../ct_mr_stdscale/val//ct/',aug=False)

In [6]:
g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()
val_loader = DataLoader(dataVal, batch_size=1, shuffle=False)
train_loader = DataLoader(dataTrain, batch_size=32, shuffle=True)

C:\Users\kiw00\AppData\Local\Temp\ipykernel_15380\595217137.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  g_scaler = torch.cuda.amp.GradScaler()
C:\Users\kiw00\AppData\Local\Temp\ipykernel_15380\595217137.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  d_scaler = torch.cuda.amp.GradScaler()


In [7]:
disc = Discriminator(in_channels=1).to('cuda')
gen = Generator(in_channels=1, out_channels=1).to('cuda')
opt_disc = optim.Adam(disc.parameters(), lr= 1e-4, betas=(0.5, 0.999),)
opt_gen = optim.Adam(gen.parameters(), lr= 5e-5, betas=(0.5, 0.999))
BCE = nn.BCEWithLogitsLoss()
L1_LOSS = nn.L1Loss()



In [8]:
best_g_loss = float('inf')
best_model_path = 'best_generator_aug.pth'

for epoch in range(50):
    train_fn(
        disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,
    )
    
    eval_metrics  = eval_metrics = eval_fn(
        gen, disc, val_loader, L1_LOSS, BCE, device='cuda'
    )

    torch.save(gen.state_dict(), best_model_path)
    # break  # remove this when running full training


  0%|          | 0/430 [00:00<?, ?it/s]C:\Users\kiw00\AppData\Local\Temp\ipykernel_15380\3934712129.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
C:\Users\kiw00\AppData\Local\Temp\ipykernel_15380\3934712129.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 430/430 [15:54<00:00,  2.22s/it, D_fake=0.448, D_loss=0.613, D_real=0.613, G_loss=3.66]   
c:\Users\kiw00\Documents\work\AIT\.venv\Lib\site-packages\torchmetrics\utilities\prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(
c:\Users\kiw00\Documents\work\AIT\.venv\Lib\site-packages\torchmetrics\utilities\prints.py:62: FutureWarning: Imp


[Validation] G_loss: 3.7701, G_fake_loss: 0.9674, L1_loss: 2.8027
[Metrics] MAE: 0.0280, PSNR: 34.07 dB, SSIM: 0.7776


100%|██████████| 430/430 [10:40<00:00,  1.49s/it, D_fake=0.469, D_loss=0.63, D_real=0.425, G_loss=3.11] 



[Validation] G_loss: 9.5015, G_fake_loss: 1.0119, L1_loss: 8.4896
[Metrics] MAE: 0.0849, PSNR: 22.58 dB, SSIM: 0.7028


100%|██████████| 430/430 [06:47<00:00,  1.06it/s, D_fake=0.305, D_loss=0.644, D_real=0.691, G_loss=3.45]


...


[Validation] G_loss: 3.2867, G_fake_loss: 1.0636, L1_loss: 2.2231
[Metrics] MAE: 0.0222, PSNR: 35.12 dB, SSIM: 0.8233


100%|██████████| 430/430 [05:44<00:00,  1.25it/s, D_fake=0.0303, D_loss=0.0957, D_real=0.868, G_loss=6.09]  



[Validation] G_loss: 3.5087, G_fake_loss: 1.3261, L1_loss: 2.1826
[Metrics] MAE: 0.0218, PSNR: 34.95 dB, SSIM: 0.8163


100%|██████████| 430/430 [05:44<00:00,  1.25it/s, D_fake=0.221, D_loss=0.855, D_real=0.223, G_loss=4.1]    



[Validation] G_loss: 4.5785, G_fake_loss: 2.4392, L1_loss: 2.1392
[Metrics] MAE: 0.0214, PSNR: 35.20 dB, SSIM: 0.8237
